# Mini Project: Hypothesis Testing in Healthcare from DataCamp
Setting: A pharmaceutical company GlobalXYZ has just completed a randomized controlled drug trial. To promote transparency and reproducibility of the drug's outcome, they (GlobalXYZ) have presented the dataset to your organization, a non-profit that focuses primarily on drug safety.

The dataset provided contained five adverse effects (headache, abdominal pain, dyspepsia, upper respiratory infection, chronic obstructive airway disease (COAD), demographic data, vital signs, lab measures, etc.), demographic data, vital signs, etc. The ratio of drug observations to placebo observations is 2 to 1. Your organization is primarily interested in the drug's adverse reactions. It wants to know if the adverse reactions, if any, are of significant proportions. It has asked you to explore and answer some questions from the data.

For this project, the dataset has been modified to reflect the presence and absence of adverse effects adverse_effects and the number of adverse effects in a single individual num_effects.

The columns in the modified dataset are:
Sex:	                (The gender of the individual), 
Age:	                (The age of the individual), 
Week:	            (The week of the drug testing), 
Trx:	                (The treatment (Drug) and control (Placebo) groups), 
Wbc:	                (The count of white blood cells), 
Rbc:	                (The count of red blood cells), 
Adverse_effects:	    (The presence of at least a single adverse effect), 
Num_effects:	        (The number of adverse effects experienced by a single individual)

The original dataset can be found here.

### Learning Objectives:

* To determine if the proportion of adverse effects differs significantly between the Drug and Placebo groups, saving the p-value as a variable called two_sample_p_value
* To find out if the number of adverse effects is independent of the treatment and control groups, saving as a variable called num_effects_p_value containing a p-value.
* To examine if there is a significant difference between the ages of the Drug and Placebo groups, storing the p-value of your test in a variable called age_group_effects_p_value


* For this analysis, I am using the drug_safety.csv dataset from Hbiostat, with acknowledgement to Vanderbilt University Department of Biostatistics.

In [ ]:
# Import packages
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import pingouin
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
drug_safety = pd.read_csv("drug_safety.csv")

# Start coding here...
drug_safety.head()




In [ ]:
adv_freq = drug_safety.groupby("trx").adverse_effects.value_counts()
success_list = adv_freq[adv_freq.index.get_level_values('adverse_effects') == "Yes"]
print(success_list)


In [ ]:
normalised_drug_effectiveness = drug_safety.groupby("trx").adverse_effects.value_counts(normalize = True)
print(normalised_drug_effectiveness)

In [ ]:
p_hat_drug = normalised_drug_effectiveness [("Drug", "Yes")]
p_hat_Placebo = normalised_drug_effectiveness[("Placebo", "Yes")]
print(p_hat_drug, p_hat_Placebo)

In [ ]:
total = drug_safety.groupby("trx")["adverse_effects"].count()
print(total)

In [ ]:
total_drug = total['Drug']
total_placebo = total['Placebo']
print(total_drug, total_placebo)

In [ ]:
p_hat = (total_drug * p_hat_drug + total_placebo * p_hat_Placebo) / (total_drug + total_placebo)

std_error = np.sqrt(p_hat * (1 - p_hat) / total_drug + p_hat * (1 - p_hat) / total_placebo)

z_score = (p_hat_drug - p_hat_Placebo) / std_error

print(z_score)

In [ ]:
n_success = np.array([1024, 512])
n_rows = np.array([10727, 5376])
from statsmodels.stats.proportion import proportions_ztest
z_score, two_sample_p_value = proportions_ztest(count=n_success, nobs=n_rows,                                   alternative="two-sided")
print(z_score, p_value)

With a z_score of 0.0452182684494942 and a two_sample_p_value of 0.9639333330262475, this shows that there is no statistically significant difference between the Drug and Placebo groups, as the p-value is much greater than the common significance level of 0.05. 

In [ ]:
import pingouin

# Perform chi-squared test of independence
expected, observed, stats = pingouin.chi2_independence(data=drug_safety, x='adverse_effects', y='trx', correction=False)

# Print the stats DataFrame
print(stats)

# Access the p-value directly from the DataFrame
num_effects_p_value = stats.loc[0, 'pval']
print(num_effects_p_value)

With num_effects_p_value = 0.9639333330262464 > 0.05 (5% significance level), this shows that the number of adverse effects is independent of the treatment and control groups — i.e., there's no statistically significant association between them.

In [ ]:
trx_age = drug_safety.loc[drug_safety["trx"] == "Drug", "age"]
placebo_age = drug_safety.loc[drug_safety["trx"] == "Placebo", "age"]
age_group_effects_p_value = pingouin.mwu(trx_age, placebo_age)["p-val"]
print(age_group_effects_p_value)

With age_group_effects_p_value = 0.256963 > 0.05 (5% significance level), this shows that there is no significant difference between the ages of the Drug and Placebo groups. 